<a href="https://colab.research.google.com/github/ashwinvaswani/Generative-Modelling-of-Images-from-Speech/blob/master/src/TIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydub
!pip install pytube
!pip install mtcnn
!pip install keras-vggface

     |████████████████████████████████| 2.3MB 1.8MB/s 


In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2 
import os
import sys
import moviepy.editor as mp
import pickle

import math
from math import ceil
from math import floor
import time

from pydub import AudioSegment
from pytube import YouTube

import shutil

import IPython.display as ipd
import librosa
import librosa.display

import dlib

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)196608/45929032 bytes (0.4%)688128/45929032 bytes (1.5%)1277952/45929032 bytes (2.8%)1982464/45929032 bytes (4.3%)2867200/45929032 bytes (6.2%)3923968/45929032 bytes (8.5%)5218304/45929032 bytes (11.4%)6742016/45929032 bytes (14.7%)8585216/45929032 bytes (18.7%)10772480/45929032 bytes (23.5%)13418496/45929032 bytes (29.2%)16621568/45929032 bytes (36.2%)20488192/45929032 bytes (

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
PATH = './drive/My Drive/TIP/Dataset/'
PATH_TO_MAIN = './drive/My Drive/TIP/'
YT_LINK = 'www.youtube.com/watch?v='

In [0]:
column_names = ['YouTube_ID', 'start_segment', 'end_segment', 'X_coordinate', 'Y_coordinate']
train_df = pd.read_csv(PATH + 'avspeech_train.csv',names = column_names)
test_df = pd.read_csv(PATH + 'avspeech_test.csv',names = column_names)

In [6]:
train_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,CJoOwXcjhds,233.266000,239.367000,0.780469,0.670833
1,AvWWVOgaMlk,90.000000,93.566667,0.586719,0.311111
2,Y8HMIm8mdns,171.607767,174.607767,0.505729,0.240741
3,akwvpAiLFk0,144.680000,150.000000,0.698438,0.288889
4,Swss72CHSWg,90.023267,97.297200,0.230729,0.204630


In [7]:
test_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,u5MPyrRJPmc,108.2400,111.240000,0.849219,0.305556
1,H1ulMfj5wRY,112.3200,116.940000,0.112500,0.345833
2,-wuxbgMRIWs,30.0300,36.745044,0.744531,0.211111
3,GNRPRH-E-sI,30.2302,38.171467,0.333594,0.494444
4,VvcwAGkSy2o,240.2000,253.366667,0.491667,0.372222


In [10]:
YT_LINK + train_df.iloc[1]["YouTube_ID"]

'www.youtube.com/watch?v=AvWWVOgaMlk'

In [13]:
shutil.rmtree('Images')
shutil.rmtree('Audio')
shutil.rmtree('Videos')
shutil.rmtree(PATH_TO_MAIN + 'Face_Images/')
os.mkdir("Images")
os.mkdir("Audio")
os.mkdir("Videos")
os.mkdir(PATH_TO_MAIN + 'Face_Images/')

len_train = train_df.shape[0]
count_unsuccessful = 0
dict_audio = {}
to_remove = []
for i in range(1,100):
  while(True):
    print(i)
    print(YT_LINK + train_df.iloc[i]["YouTube_ID"])
    try:
      yt = YouTube(YT_LINK + train_df.iloc[i]["YouTube_ID"])
      stream = yt.streams.first()
      stream.download(output_path = 'Videos/',filename = train_df.iloc[i]["YouTube_ID"])


      cap = cv2.VideoCapture("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mp4') 
      fps = cap.get(cv2.CAP_PROP_FPS)
      
      if fps == 0:
        continue

      count = 0

      dir_name = train_df.iloc[i]["YouTube_ID"]
      
      print("FPS is : "  + str(fps))
      start_cnt = train_df.iloc[i]["start_segment"]
      end_cnt = train_df.iloc[i]["end_segment"]
      
      while(1):

          ret,frame = cap.read()

          if ret == 1:

            count += 1

            (h, w) = frame.shape[:2]

            if (((start_cnt + end_cnt)*fps)/2) - 16 < count < (((start_cnt + end_cnt)*fps)/2) + 16:
              dnnFaceDetector = dlib.cnn_face_detection_model_v1( PATH_TO_MAIN + "Face_detection/mmod_human_face_detector.dat")
              faceRects = dnnFaceDetector(frame, 0)
              print(faceRects)

              if len(faceRects) == 0:
                to_remove.append(dir_name)
                continue
              for faceRect in faceRects:
                  x1 = faceRect.rect.left()
                  y1 = faceRect.rect.top()
                  x2 = faceRect.rect.right()
                  y2 = faceRect.rect.bottom()
                  img = frame[y1:y2, x1:x2]
                  print("About to crop")
                  print(count)
                  img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
                  cv2.imwrite('Images/' +dir_name + '.png', img)
                  cv2.imwrite(PATH_TO_MAIN + 'Face_Images/' +dir_name + '.png', img)
              break


            if count > int(end_cnt*fps):
              break
            k = cv2.waitKey(1)
            if k == ord('q'):
              break

      cap.release()
      cv2.destroyAllWindows()
      time.sleep(5)
    except:
      print("SORRY!!! The video is unavailable")
      time.sleep(5)
      count_unsuccessful += 1
      continue
      
    try:
      clip = mp.VideoFileClip("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mp4').subclip(start_cnt,end_cnt)
      clip.audio.write_audiofile("Audio/" + train_df.iloc[i]["YouTube_ID"] +'.mp3') 
      print("before dict")
      dict_audio[train_df.iloc[i]["YouTube_ID"]] = end_cnt - start_cnt
    except:
      print("Error with audio")  
      continue
    
    break

1
www.youtube.com/watch?v=AvWWVOgaMlk
SORRY!!! The video is unavailable
1
www.youtube.com/watch?v=AvWWVOgaMlk
SORRY!!! The video is unavailable
1
www.youtube.com/watch?v=AvWWVOgaMlk
SORRY!!! The video is unavailable


KeyboardInterrupt: ignored

In [0]:
# Preparing y_train for encoder and x_train for decoder

In [0]:
def get_embeddings(filenames):

  faces = []
  for f in filenames:
    img = cv2.imread(f)
    face_arr = asarray(img)
    faces.append(face_arr)

  samples = asarray(faces,'float32')

  samples = preprocess_input(samples,version=2)

  model = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')

  yhat = model.predict(samples)

  return yhat

In [12]:
face_list = []
for elem in os.listdir('./Images'):
  face_list.append('./Images/' + elem)
y_train_encoder = get_embeddings(face_list)
x_train_decoder = get_embeddings(face_list)














94699520/94694792 [==============================] - 3s 0us/step


In [13]:
y_train_encoder.shape

(16, 2048)

In [0]:
# Using y_train to create decoder

In [0]:
# Input is y_train and output is images from Images folder

In [0]:
# Augmenting Audio

In [0]:
# shutil.rmtree('Processed_Audio')
os.mkdir('Processed_Audio')
temp_dict = dict_audio.copy()
for elem in os.listdir('./Audio'):
  sound = AudioSegment.from_mp3('./Audio/' + elem)
  sound_new = sound
  while temp_dict[elem[:-4]] < 6:
    print("here")
    sound_new += sound
    temp = temp_dict[elem[:-4]]
    temp_dict[elem[:-4]] = 2*temp
    sound = sound_new

  extract = sound[0:6*1000]

  extract.export('Processed_Audio/' + elem, format="mp3")


In [0]:
# to_remove = ['2f32XSMYlDk']
for elem in to_remove:
  os.remove('Processed_Audio/' + elem + '.mp3')

In [0]:
# Preparing x_train

In [0]:
# shutil.rmtree('Spectograms')
os.mkdir('Spectograms')
for filename in os.listdir("./Processed_Audio"):
  audio_path = './Processed_Audio/' + filename
  x , sr = librosa.load(audio_path,sr = 15925)
  print(audio_path)
  print(type(x), type(sr))
  print(x.shape, sr)
  ipd.Audio(audio_path)
  window_size = 25
  window = np.hanning(window_size)
  h1 = 160
  print(h1)

  stft  = librosa.core.spectrum.stft(x, n_fft=512, hop_length=h1,win_length=window_size, window=window)
  out = 2 * np.abs(stft) / np.sum(window_size)
  with open('./Spectograms/' + filename[:-4] + '.pkl','wb') as f:
    pickle.dump(stft,f)
  
  # librosa.display.waveplot(x, sr=sr)
  # mfccs = librosa.feature.mfcc(x, sr=sr)
  # print(mfccs.shape)
  # librosa.display.specshow(mfccs, sr=sr, x_axis='time')
  # print()




In [0]:
stft.shape

In [0]:
for elem in stft:
  print(elem)

In [0]:
# Todo: For all pickle files in spectogram 

In [0]:
def get_encoder_network_input(k):
  shape_x = k.shape[0]
  shape_y = k.shape[1]
  mod = np.zeros((shape_x,shape_y))
  for i in range(shape_x):
    for j in range(shape_y):
      mod[i][j] = np.abs(k[i][j])

  theta_ = np.zeros((shape_x,shape_y))
  for i in range(shape_x):
    for j in range(shape_y):
      theta_[i][j] = np.angle(k[i][j])

  real = np.zeros((shape_x,shape_y))
  complex_ = np.zeros((shape_x,shape_y))
  for i in range(shape_x):
    for j in range(shape_y):
      temp = mod[i][j]
      real[i][j] = temp * math.cos(complex_[i][j])
      complex_[i][j] = temp * math.sin(complex_[i][j])

  real = np.expand_dims(real,axis = 0)
  complex_ = np.expand_dims(complex_,axis = 0)
  combined = np.concatenate((real,complex_))

  return combined

In [0]:
encoder_input = []
for elem in os.listdir('./Spectograms/'):
  with open('./Spectograms/' + elem,'rb') as f:
    print('./Spectograms/' + elem)
    k = pickle.load(f)
    print(k.shape)

  encoder_input.append(np.transpose(get_encoder_network_input(k)))

In [0]:
len(encoder_input)

In [0]:
# For encoder

# Input : encoder input as np array
# Output : y_train_encoder

# FOr decoder

# Input : x_train_encoder
# Output : Images from Images folder

In [0]:
x_enc_train = np.asarray(encoder_input)

In [0]:
x_enc_train.shape

In [0]:
y_train_encoder.shape

In [0]:
with open(PATH_TO_MAIN + 'pickles/encoder_trainX_2500.pkl','wb') as f:
    pickle.dump(x_enc_train,f)

In [0]:
with open(PATH_TO_MAIN + 'pickles/encoder_trainY_2500.pkl','wb') as f:
    pickle.dump(y_train_encoder,f)

In [0]:
# from google.colab import files
# files.download('encoder_trainY.pkl')

In [0]:
!nvidia-smi